In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#!pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import datetime as dt
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
import statsmodels.api as sm
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from fbprophet import Prophet
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.stattools import adfuller


std=StandardScaler()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-19-datacsv/covid_19_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths_US.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed_US.csv
/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv


In [2]:
covid=pd.read_csv('../input/covid-19-datacsv/covid_19_data.csv')
covid
india_data=covid[covid['Country/Region']=='South Korea']
india_data['ObservationDate']=pd.to_datetime(india_data['ObservationDate'])

india_data


,Unnamed: 0,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
37,37,38,2020-01-22,NaN,South Korea,1/22/2020 17:00,1.0,0.0,0.0
75,75,76,2020-01-23,NaN,South Korea,1/23/20 17:00,1.0,0.0,0.0
121,121,122,2020-01-24,NaN,South Korea,1/24/20 17:00,2.0,0.0,0.0
162,162,163,2020-01-25,NaN,South Korea,1/25/20 17:00,2.0,0.0,0.0
208,208,209,2020-01-26,NaN,South Korea,1/26/20 16:00,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
232341,232341,232342,2021-02-23,NaN,South Korea,2021-02-24 05:29:16,88120.0,1576.0,79050.0
233105,233105,233106,2021-02-24,NaN,South Korea,2021-02-25 05:24:57,88516.0,1581.0,79487.0
233869,233869,233870,2021-02-25,NaN,South Korea,2021-02-26 05:22:40,88922.0,1585.0,79880.0
234633,234633,234634,2021-02-26,NaN,South Korea,2021-02-27 05:22:28,89321.0,1595.0,80333.0


In [3]:

datawise_India=india_data.groupby('ObservationDate').agg({'Confirmed':'sum','Recovered':'sum','Deaths':'sum'})
datawise_India
print(datawise_India.iloc[-1])

Total_active=datawise_India['Confirmed'].iloc[-1]-datawise_India['Recovered'].iloc[-1]-datawise_India['Deaths'].iloc[-1]
Total_closed=datawise_India['Recovered'].iloc[-1]+datawise_India['Deaths'].iloc[-1]

print('Total_active:',Total_active)
print('Total_closed:',Total_closed)

Confirmed    89676.0
Recovered    80697.0
Deaths        1603.0
Name: 2021-02-27 00:00:00, dtype: float64
Total_active: 7376.0
Total_closed: 82300.0


In [4]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=datawise_India.index,y=datawise_India['Confirmed'],mode='lines+markers',name='Confirmed Cases'))
fig.add_trace(go.Scatter(x=datawise_India.index,y=datawise_India['Recovered'],mode='lines+markers',name='Recovered Cases'))
fig.add_trace(go.Scatter(x=datawise_India.index,y=datawise_India['Deaths'],mode='lines+markers',name='Deaths Cases'))
fig.update_layout(title='Growth of different types of cases in India',xaxis_title='Date',yaxis_title='Number of Cases',
                  legend=dict(x=0,y=1,traceorder='normal'))
fig.show()

In [5]:
fig=px.bar(x=datawise_India.index,y=datawise_India['Confirmed']-datawise_India['Recovered']-datawise_India['Deaths'])
fig.update_layout(title='Active Cases in India',xaxis_title='Date',yaxis_title='Number of Cases')
fig.show()

In [6]:
Growth_confirmed=[]
Growth_Recovered=[]
Growth_Deaths=[]
for i in range(datawise_India.shape[0]-1):
    Growth_confirmed.append(datawise_India['Confirmed'].iloc[i+1]/datawise_India['Confirmed'].iloc[i])
    Growth_Recovered.append(datawise_India['Recovered'].iloc[i+1]/datawise_India['Recovered'].iloc[i])
    Growth_Deaths.append(datawise_India['Deaths'].iloc[i+1]/datawise_India['Deaths'].iloc[i])
fig=go.Figure()
fig.add_trace(go.Scatter(x=datawise_India.index,y=Growth_confirmed,mode='lines',name='Growth factor of Confirmed Cases'))
fig.add_trace(go.Scatter(x=datawise_India.index,y=Growth_Recovered, mode='lines',name='Growth factor of Recovered Cases'))
fig.add_trace(go.Scatter(x=datawise_India.index,y=Growth_Recovered, mode='lines',name='Growth factor of Recovered Cases'))

fig.update_layout(title='Growth factor of different cases', xaxis_title='Date',yaxis_title='Growth factor')

fig.show()

In [7]:
datawise=india_data.groupby('ObservationDate').agg({'Confirmed':'sum','Recovered':'sum','Deaths':'sum'})
datawise['Since day']=datawise_India.index-datawise_India.index[0]
datawise['Since day']=datawise['Since day'].dt.days
set_data=int(datawise.shape[0]*0.95)
train_data=datawise.iloc[:set_data]
valid_data=datawise.iloc[set_data:]

train_data

,Confirmed,Recovered,Deaths,Since day
ObservationDate,,,,
2020-01-22,1.0,0.0,0.0,0
2020-01-23,1.0,0.0,0.0,1
2020-01-24,2.0,0.0,0.0,2
2020-01-25,2.0,0.0,0.0,3
2020-01-26,3.0,0.0,0.0,4
...,...,...,...,...
2021-02-02,79311.0,69299.0,1441.0,377
2021-02-03,79762.0,69704.0,1448.0,378
2021-02-04,80131.0,70117.0,1459.0,379


In [8]:
linear_reg=LinearRegression(normalize=True)

linear_reg.fit(np.array(train_data['Since day']).reshape(-1,1),np.array(train_data['Confirmed']).reshape(-1,1))
prediction_linear_reg=linear_reg.predict(np.array(valid_data['Since day']).reshape(-1,1))

model_scores=[]
model_scores.append(np.sqrt(mean_squared_error(valid_data['Confirmed'],prediction_linear_reg)))
model_scores

[29967.54285922497]

In [9]:
line_output=[]
prediction_line=linear_reg.predict(np.array(datawise['Since day']).reshape(-1,1))
for i in range(prediction_line.shape[0]):
    line_output.append(prediction_line[i][0])
    
fig=go.Figure()
fig.add_trace(go.Scatter(x=datawise.index,y=line_output,mode='lines',name='linear regression prediction'))
fig.add_trace(go.Scatter(x=datawise.index,y=datawise['Confirmed'],mode='lines',name='Confirmed cases'))
fig.update_layout(title='Confirmed Cases linear regeression prediction',xaxis_title='Date',yaxis_title='Number of cases')
fig.show()

In [10]:
poly=PolynomialFeatures(8)
train_poly=poly.fit_transform(np.array(train_data['Since day']).reshape(-1,1))
valid_poly=poly.fit_transform(np.array(valid_data['Since day']).reshape(-1,1))
y=train_data['Confirmed']

linear_reg=LinearRegression(normalize=True)
linear_reg.fit(train_poly,y)

prediction_linear=linear_reg.predict(valid_poly)

rsme_poly=np.sqrt(mean_squared_error(valid_data['Confirmed'],prediction_linear))
model_scores.append(rsme_poly)

model_scores

[29967.54285922497, 34038.71959289509]

In [11]:
line_output=[]
poly_data=poly.fit_transform(np.array(datawise['Since day']).reshape(-1,1))
prediction_line=linear_reg.predict(poly_data)


fig=go.Figure()
fig.add_trace(go.Scatter(x=datawise.index,y=prediction_line,mode='lines',name='Polynomial regression prediction'))
fig.add_trace(go.Scatter(x=datawise.index,y=datawise['Confirmed'],mode='lines',name='Confirmed cases'))
fig.update_layout(title='Confirmed Cases polynomial regeression prediction',xaxis_title='Date',yaxis_title='Number of cases')
fig.show()

In [12]:
!pip install pmdarima

     |████████████████████████████████| 1.5 MB 412 kB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 6.6 MB/s eta 0:00:01
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21


In [13]:
from pmdarima import auto_arima
model=auto_arima(train_data['Confirmed'],start_p=0,start_q=0,max_p=4,max_q=0,
                 suppress_warnings=True,stepwise=False,seasonal=False)
                  
model.fit(train_data["Confirmed"])
y_prediction=valid_data.copy()

In [14]:
prediction=model.predict(len(valid_data))
y_prediction['ar']=prediction

prediction

array([81264.07090085, 81640.48614148, 82014.29090865, 82388.50674298,
       82765.17786123, 83142.11862837, 83519.9618423 , 83899.06725114,
       84279.00882572, 84659.92050675, 85041.86540669, 85424.76091704,
       85808.63518369, 86193.49913034, 86579.33678915, 86966.15402744,
       87353.95270149, 87742.7297338 , 88132.48633875, 88523.2228242 ,
       88914.93859876])

In [15]:
model_scores.append(np.sqrt(mean_squared_error(valid_data['Confirmed'],prediction)))
model_scores

[29967.54285922497, 34038.71959289509, 544.0264968150233]

In [16]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=valid_data.index,y=prediction,mode='lines+markers',name='AR prediction'))
fig.add_trace(go.Scatter(x=datawise.index,y=datawise['Confirmed'],mode='lines',name='Confirmed cases'))
fig.update_layout(title='Confirmed Cases AR prediction',xaxis_title='Date',yaxis_title='Number of cases')
fig.show()

In [17]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)
new_ARprediction=[]
for i in range(1,21):
    new_ARprediction.append(model.predict(len(valid_data)+i)[-1])
    
new_ARprediction
new_date=[]
for i in range(1,21):
    new_date.append(datawise.index[-1]+timedelta(days=i))
model_prediction=pd.DataFrame(data=new_ARprediction,index=new_date,columns=['AR'])
model_prediction

,AR
2021-02-28,89307.633912
2021-03-01,89701.308814
2021-03-02,90095.963190
2021-03-03,90491.597093
2021-03-04,90888.210529
2021-03-05,91285.803477
2021-03-06,91684.375948
2021-03-07,92083.927942
2021-03-08,92484.459455
2021-03-09,92885.970489


In [18]:
prediction=model.predict(int(20))
prediction

array([81264.07090085, 81640.48614148, 82014.29090865, 82388.50674298,
       82765.17786123, 83142.11862837, 83519.9618423 , 83899.06725114,
       84279.00882572, 84659.92050675, 85041.86540669, 85424.76091704,
       85808.63518369, 86193.49913034, 86579.33678915, 86966.15402744,
       87353.95270149, 87742.7297338 , 88132.48633875, 88523.2228242 ])

In [19]:
model=auto_arima(datawise['Confirmed'],start_p=0,start_q=0,max_p=4,max_q=0,
                 suppress_warnings=True,stepwise=False,seasonal=False)
model.fit(datawise['Confirmed'])
prediction=[]
for i in range(1,21):
    prediction.append(model.predict(len(valid_data)+i)[-1])


model_prediction2=pd.DataFrame(data=prediction,index=new_date,columns=['AR'])
model_prediction2

,AR
2021-02-28,98152.981527
2021-03-01,98549.132971
2021-03-02,98946.222832
2021-03-03,99344.250027
2021-03-04,99743.216298
2021-03-05,100143.120620
2021-03-06,100543.963109
2021-03-07,100945.744117
2021-03-08,101348.463290
2021-03-09,101752.120778


In [20]:
model=auto_arima(train_data['Confirmed'],start_p=0,start_q=0,max_p=0,max_q=4)
model.fit(train_data['Confirmed'])

ARIMA(order=(0, 2, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [21]:
prediction=model.predict(len(valid_data))
y_prediction['MA']=prediction

In [22]:
model_scores.append(np.sqrt(mean_squared_error(valid_data['Confirmed'],prediction)))
model_scores

[29967.54285922497, 34038.71959289509, 544.0264968150233, 552.0898187123124]

In [23]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=valid_data.index,y=prediction,mode='lines+markers',name='MA prediction'))
fig.add_trace(go.Scatter(x=datawise.index,y=datawise['Confirmed'],mode='lines',name='Confirmed cases'))
fig.update_layout(title='Confirmed Cases MA prediction',xaxis_title='Date',yaxis_title='Number of cases')
fig.show()

In [24]:
new_MAprediction=[]
for i in range(1,21):
    new_MAprediction.append(model.predict(len(valid_data)+i)[-1])
    
model_prediction['MA']=new_MAprediction

model_prediction

,AR,MA
2021-02-28,89307.633912,89235.326756
2021-03-01,89701.308814,89614.387063
2021-03-02,90095.963190,89993.447370
2021-03-03,90491.597093,90372.507677
2021-03-04,90888.210529,90751.567984
2021-03-05,91285.803477,91130.628291
2021-03-06,91684.375948,91509.688598
2021-03-07,92083.927942,91888.748905
2021-03-08,92484.459455,92267.809212
2021-03-09,92885.970489,92646.869519


In [25]:
model=auto_arima(datawise['Confirmed'],start_p=0,start_q=0,max_p=0,max_q=4)
model.fit(datawise['Confirmed'])
prediction=[]
for i in range(1,21):
    prediction.append(model.predict(len(valid_data)+i)[-1])
    
model_prediction2['MA']=prediction
model_prediction2

,AR,MA
2021-02-28,98152.981527,97835.590963
2021-03-01,98549.132971,98206.839370
2021-03-02,98946.222832,98578.087777
2021-03-03,99344.250027,98949.336184
2021-03-04,99743.216298,99320.584591
2021-03-05,100143.120620,99691.832998
2021-03-06,100543.963109,100063.081405
2021-03-07,100945.744117,100434.329812
2021-03-08,101348.463290,100805.578219
2021-03-09,101752.120778,101176.826626


In [26]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=new_date,y=prediction,mode='lines+markers',name='MA prediction'))
fig.add_trace(go.Scatter(x=datawise.index,y=datawise['Confirmed'],mode='lines',name='Confirmed cases'))
fig.update_layout(title='Confirmed Cases MA prediction',xaxis_title='Date',yaxis_title='Number of cases')
fig.show()

In [27]:
model=auto_arima(train_data['Confirmed'],trace=True, error_action='ignore',start_p=1,start_q=1,max_p=3,max_q=3,
                 suppress_warnings=True,stepwise=False,seasonal=False)
model.fit(train_data['Confirmed'])

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=4333.300, Time=0.02 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=4296.456, Time=0.09 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=4298.230, Time=0.25 sec
 ARIMA(0,2,3)(0,0,0)[0] intercept   : AIC=4293.506, Time=0.35 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=4305.109, Time=0.13 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=4298.339, Time=0.15 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=4296.180, Time=0.39 sec
 ARIMA(1,2,3)(0,0,0)[0] intercept   : AIC=4294.497, Time=0.55 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=4292.008, Time=0.15 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=4289.902, Time=0.22 sec
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=4291.051, Time=0.69 sec
 ARIMA(2,2,3)(0,0,0)[0] intercept   : AIC=4291.856, Time=1.40 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=4290.488, Time=0.28 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=4291.565, Time=0.54 sec
 ARIMA(3,2,2)(0,0,0)[0] intercept   : AIC=4292.824, Time=0.88 sec

Best mode

ARIMA(order=(2, 2, 1), scoring_args={}, suppress_warnings=True)

In [28]:
prediction=model.predict(len(valid_data))
y_prediction['ARIMA']=prediction
model_scores.append(np.sqrt(mean_squared_error(valid_data['Confirmed'],prediction)))
model_scores

[29967.54285922497,
 34038.71959289509,
 544.0264968150233,
 552.0898187123124,
 503.38079680363694]

In [29]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=valid_data.index,y=prediction,mode='lines+markers',name='ARIMA prediction'))
fig.add_trace(go.Scatter(x=datawise.index,y=datawise['Confirmed'],mode='lines',name='Confirmed cases'))
fig.update_layout(title='Confirmed Cases ARIMA prediction',xaxis_title='Date',yaxis_title='Number of cases')
fig.show()

In [30]:
new_ARIMAprediction=[]
for i in range(1,21):
    new_ARIMAprediction.append(model.predict(len(valid_data)+i)[-1])
    

prediction=model.predict(20)

model_prediction['ARIMA']=new_ARIMAprediction

model_prediction

,AR,MA,ARIMA
2021-02-28,89307.633912,89235.326756,89384.078462
2021-03-01,89701.308814,89614.387063,89781.310878
2021-03-02,90095.963190,89993.447370,90179.528942
2021-03-03,90491.597093,90372.507677,90578.732619
2021-03-04,90888.210529,90751.567984,90978.921921
2021-03-05,91285.803477,91130.628291,91380.096851
2021-03-06,91684.375948,91509.688598,91782.257403
2021-03-07,92083.927942,91888.748905,92185.403581
2021-03-08,92484.459455,92267.809212,92589.535383
2021-03-09,92885.970489,92646.869519,92994.652809


In [31]:
model=auto_arima(datawise['Confirmed'],trace=True, error_action='ignore',start_p=1,start_q=1,max_p=3,max_q=3,
                 suppress_warnings=True,stepwise=False,seasonal=False)
model.fit(datawise['Confirmed'])
prediction=[]
for i in range(1,21):
    prediction.append(model.predict(len(valid_data)+i)[-1])
    
    
model_prediction2['ARIMA']=prediction
model_prediction2

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=4574.706, Time=0.02 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=4544.365, Time=0.08 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=4544.292, Time=0.25 sec
 ARIMA(0,2,3)(0,0,0)[0] intercept   : AIC=4542.217, Time=0.32 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=4552.526, Time=0.08 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=4545.163, Time=0.22 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=4542.063, Time=0.24 sec
 ARIMA(1,2,3)(0,0,0)[0] intercept   : AIC=4542.881, Time=0.71 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=4540.423, Time=0.16 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=4538.850, Time=0.30 sec
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=4540.318, Time=0.80 sec
 ARIMA(2,2,3)(0,0,0)[0] intercept   : AIC=4541.952, Time=1.91 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=4540.303, Time=0.34 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=4540.747, Time=0.46 sec
 ARIMA(3,2,2)(0,0,0)[0] intercept   : AIC=4542.730, Time=0.71 sec

Best mode

,AR,MA,ARIMA
2021-02-28,98152.981527,97835.590963,98045.380028
2021-03-01,98549.132971,98206.839370,98436.266476
2021-03-02,98946.222832,98578.087777,98828.073754
2021-03-03,99344.250027,98949.336184,99220.801849
2021-03-04,99743.216298,99320.584591,99614.450770
2021-03-05,100143.120620,99691.832998,100009.020514
2021-03-06,100543.963109,100063.081405,100404.511081
2021-03-07,100945.744117,100434.329812,100800.922471
2021-03-08,101348.463290,100805.578219,101198.254685
2021-03-09,101752.120778,101176.826626,101596.507722


In [32]:
model=auto_arima(train_data['Confirmed'],trace=True, error_action='ignore',start_p=1,start_q=1,max_p=3,max_q=3,
                 suppress_warnings=True,stepwise=True,seasonal=True)
model.fit(train_data['Confirmed'])

Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=4296.526, Time=0.06 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=4331.370, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=4303.235, Time=0.04 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=4294.639, Time=0.04 sec
 ARIMA(0,2,2)(0,0,0)[0]             : AIC=4296.419, Time=0.08 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=4294.376, Time=0.16 sec
 ARIMA(2,2,2)(0,0,0)[0]             : AIC=4289.199, Time=0.25 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=4288.071, Time=0.16 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=4290.197, Time=0.08 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=4289.718, Time=0.20 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=4288.639, Time=0.09 sec
 ARIMA(3,2,2)(0,0,0)[0]             : AIC=4290.937, Time=0.35 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=4289.902, Time=0.25 sec

Best model:  ARIMA(2,2,1)(0,0,0)[0]          
Total fit time: 1.809 seconds


ARIMA(order=(2, 2, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [33]:
prediction=model.predict(len(valid_data))
y_prediction['SARIMA']=prediction
model_scores.append(np.sqrt(mean_squared_error(valid_data['Confirmed'],prediction)))
model_scores

[29967.54285922497,
 34038.71959289509,
 544.0264968150233,
 552.0898187123124,
 503.38079680363694,
 610.9722579330365]

In [34]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=valid_data.index,y=prediction,mode='lines+markers',name='SARIMA prediction'))
fig.add_trace(go.Scatter(x=datawise.index,y=datawise['Confirmed'],mode='lines',name='Confirmed cases'))
fig.update_layout(title='Confirmed Cases SARIMA prediction',xaxis_title='Date',yaxis_title='Number of cases')
fig.show()

In [35]:
new_SARIMAprediction=[]
for i in range(1,21):
    new_SARIMAprediction.append(model.predict(len(valid_data)+i)[-1])
    
model_prediction['SARIMA']=new_SARIMAprediction

model_prediction

,AR,MA,ARIMA,SARIMA
2021-02-28,89307.633912,89235.326756,89384.078462,89126.528341
2021-03-01,89701.308814,89614.387063,89781.310878,89500.723326
2021-03-02,90095.963190,89993.447370,90179.528942,89874.918334
2021-03-03,90491.597093,90372.507677,90578.732619,90249.113331
2021-03-04,90888.210529,90751.567984,90978.921921,90623.308329
2021-03-05,91285.803477,91130.628291,91380.096851,90997.503329
2021-03-06,91684.375948,91509.688598,91782.257403,91371.698328
2021-03-07,92083.927942,91888.748905,92185.403581,91745.893327
2021-03-08,92484.459455,92267.809212,92589.535383,92120.088326
2021-03-09,92885.970489,92646.869519,92994.652809,92494.283325


In [36]:
model=auto_arima(datawise['Confirmed'],trace=True, error_action='ignore',start_p=1,start_q=1,max_p=3,max_q=3,
                 suppress_warnings=True,stepwise=True,seasonal=True)
model.fit(datawise['Confirmed'])
prediction=[]
for i in range(1,21):
    prediction.append(model.predict(len(valid_data)+i)[-1])
    
model_prediction2['SARIMA']=prediction
model_prediction2

Performing stepwise search to minimize aic
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=4543.327, Time=0.08 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=4572.766, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=4550.629, Time=0.02 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=4542.513, Time=0.04 sec
 ARIMA(0,2,2)(0,0,0)[0]             : AIC=4542.462, Time=0.13 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=4540.232, Time=0.18 sec
 ARIMA(2,2,2)(0,0,0)[0]             : AIC=4538.430, Time=0.42 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=4536.990, Time=0.23 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=4538.579, Time=0.09 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=4538.878, Time=0.18 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=4538.436, Time=0.09 sec
 ARIMA(3,2,2)(0,0,0)[0]             : AIC=4540.878, Time=0.26 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=4538.850, Time=0.35 sec

Best model:  ARIMA(2,2,1)(0,0,0)[0]          
Total fit time: 2.116 seconds


,AR,MA,ARIMA,SARIMA
2021-02-28,98152.981527,97835.590963,98045.380028,97805.093383
2021-03-01,98549.132971,98206.839370,98436.266476,98174.471477
2021-03-02,98946.222832,98578.087777,98828.073754,98543.849578
2021-03-03,99344.250027,98949.336184,99220.801849,98913.227673
2021-03-04,99743.216298,99320.584591,99614.450770,99282.605770
2021-03-05,100143.120620,99691.832998,100009.020514,99651.983867
2021-03-06,100543.963109,100063.081405,100404.511081,100021.361963
2021-03-07,100945.744117,100434.329812,100800.922471,100390.740060
2021-03-08,101348.463290,100805.578219,101198.254685,100760.118157
2021-03-09,101752.120778,101176.826626,101596.507722,101129.496253
